In [10]:
import tkinter as tk
import mysql.connector as connector

def connect_to_database():
    global connection
    try:
        # Retrieve the password from the entry widget
        password = password_entry.get()

        # Establish connection between Python and MySQL database via connector API
        connection = connector.connect(host="localhost",
                                       user="DisenchantmentAdmin", # use your own
                                       password=password, # use the entered password
                                       raise_on_warnings=True)
        print("Connection between MySQL and Python is established.\n")
        connection_label.config(text="Connected")
        open_second_window()
    except connector.Error as err:
        print(f"Error connecting to the database: {err}")

def close_connection():
    global connection
    try:
        # Close the database connection
        connection.close()
        print("Connection to the database is closed.\n")
        connection_label.config(text="Not Connected")
    except connector.Error as err:
        print(f"Error closing the connection: {err}")

def open_second_window():
    second_window = tk.Toplevel()
    second_window.title("Second Window")

    # Create the "Elements" button in the second window
    elements_button = tk.Button(second_window, text="Elements")
    elements_button.pack()

# Create the main window
window = tk.Tk()

# Create the password entry widget
password_entry = tk.Entry(window, show="*")
password_entry.pack()

# Create the "Connect" button
connect_button = tk.Button(window, text="Connect", command=connect_to_database)
connect_button.pack()

# Create the "Close Connection" button
close_button = tk.Button(window, text="Close Connection", command=close_connection)
close_button.pack()

# Create the connection status label
connection_label = tk.Label(window, text="Not Connected")
connection_label.pack()

# Start the Tkinter event loop
window.mainloop()


Connection between MySQL and Python is established.

Connection to the database is closed.

